In [1]:
from utils.models import run_rsf_model, run_cox_model, plot_cox_coefficients, run_metrics, cross_validate_and_plot, \
    plot_model_residuals, plot_coefficients_rsf, save_rsf_model_output
from utils.config import load_datasets, get_generated_gan_train_dataset_indexes, get_test_dataset_indexes, \
    get_generated_vae_dataset_indexes, get_generated_timevary_dataset_indexes, get_train_dataset_indexes
from utils.preprocess import preprocess

In [2]:
test_folder = "../outputs/test_sets"
test_indexes = get_test_dataset_indexes(test_folder)
test_sets = load_datasets(folder=test_folder, names=test_indexes)

sim_folder = "../outputs/generated_datasets"
folder = "../outputs/datasets"
gan_dataset_indexes = get_generated_gan_train_dataset_indexes(sim_folder)
vae_dataset_indexes = get_generated_vae_dataset_indexes(sim_folder)
# timevary_dataset_indexes = get_generated_timevary_dataset_indexes(sim_folder)
real_dataset_indexes = get_train_dataset_indexes(folder)
gan_datasets = load_datasets(folder=sim_folder, names=gan_dataset_indexes) 
vae_datasets = load_datasets(folder=sim_folder, names=vae_dataset_indexes) 
# timevary_datasets = load_datasets(folder=sim_folder, names=timevary_dataset_indexes) 
real_datasets = load_datasets(folder=folder, names=real_dataset_indexes) 

In [3]:
gan = gan_datasets['colon_gan']
vae = vae_datasets['colon_vae']
# time = timevary_datasets['csl_timegan']
real = preprocess(real_datasets['colon'])
test = preprocess(test_sets['colon'])

Categorical columns selected: ['fac_rx', 'fac_sex', 'fac_differ', 'fac_obstruct', 'fac_perfor', 'fac_adhere', 'fac_node4']
Numerical columns selected: ['num_age', 'num_nodes']
Number of transformers: 3
Transformers: [('ohe', Pipeline(steps=[('ohe',
                 OneHotEncoder(drop='first', handle_unknown='ignore',
                               sparse=False, sparse_output=False))]), ['fac_rx', 'fac_sex', 'fac_differ', 'fac_obstruct', 'fac_perfor', 'fac_adhere', 'fac_node4']), ('imp', Pipeline(steps=[('imp', SimpleImputer(strategy='median')),
                ('scale', StandardScaler())]), ['num_age', 'num_nodes']), ('remainder', 'passthrough', [0, 1, 2])]
Categorical columns selected: ['fac_rx', 'fac_sex', 'fac_differ', 'fac_obstruct', 'fac_perfor', 'fac_adhere', 'fac_node4']
Numerical columns selected: ['num_age', 'num_nodes']
Number of transformers: 3
Transformers: [('ohe', Pipeline(steps=[('ohe',
                 OneHotEncoder(drop='first', handle_unknown='ignore',
               

C:\Users\johan\anaconda3\envs\reseach_project_models_env\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\johan\anaconda3\envs\reseach_project_models_env\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
real

,fac_rx_Lev+5FU,fac_rx_Obs,fac_sex_M,fac_differ_moderate,fac_differ_poor,fac_differ_well,fac_obstruct_Y,fac_perfor_Y,fac_adhere_Y,fac_node4_Y,num_age,num_nodes,pid,event,time
203,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.731360,0.092492,204.0,1.0,2552.0
856,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.541860,-0.186705,857.0,0.0,2070.0
800,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.307783,2.326064,801.0,1.0,355.0
412,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.260717,-0.186705,413.0,0.0,2234.0
721,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.626824,0.092492,722.0,0.0,2303.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.626824,-0.745098,64.0,0.0,2739.0
396,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.987497,-0.745098,397.0,0.0,2691.0
798,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.732604,-0.745098,799.0,0.0,1929.0
782,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.752176,-0.745098,783.0,1.0,832.0


In [5]:
gan_cox_model, function_data_gan, _,_,_,_= run_cox_model(gan, test)
vae_cox_model,function_data_vae, _,_,_,_ = run_cox_model(vae, test)
cox_model,function_data,_,_,_,_ = run_cox_model(real, test)

The ``p_value_threshold`` is set at 0.01. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 5000 total observations, 2512 right-censored observations>
         test_name = proportional_hazard_test

---
                          test_statistic      p  -log2(p)
fac_adhere_Y        km              6.73   0.01      6.72
                    rank            6.78   0.01      6.76
fac_differ_moderate km              1.38   0.24      2.06
                    rank            1.37   0.24      2.05
fac_differ_poor     km             45.31 <0.005     35.79
                    rank           45.39 <0.005     35.85
fac_differ_well     km             15.53 <0.005     13.59
                    rank           15.52 <0.005     13.58
fac_node4_Y         km            184.98 <0.005    137.53
                    rank          185.14 <0.005    137.65
fac_obstruct_Y      km             58.23 <0.005     45.28
                    rank           58.35 <0.005     45.37
fac_perfor_Y        km              0.02   0.89      0.16
                    rank            0.02   0.89      0.16
fac_rx_Lev+5FU      km              0.20   0.66      0.61
                    rank            0.19   0.66      0.59
fac_rx_Obs          km              6.98   0.01      6.92
                    rank            6.92   0.01      6.88
fac_sex_M           km             64.59 <0.005     49.95
                    rank           64.63 <0.005     49.98
num_age             km             61.56 <0.005     47.73
                    rank           61.69 <0.005     47.82
num_nodes           km              3.75   0.05      4.24
                    rank            3.74   0.05      4.24
pid                 km            204.51 <0.005    151.69
                    rank          204.17 <0.005    151.45



1. Variable 'fac_rx_Obs' failed the non-proportional test: p-value is 0.0083.

   Advice: with so few unique values (only 2), you can include `strata=['fac_rx_Obs', ...]` in the
call in `.fit`. See documentation in link [E] below.

2. Variable 'fac_sex_M' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['fac_sex_M', ...]` in the
call in `.fit`. See documentation in link [E] below.

3. Variable 'fac_differ_poor' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['fac_differ_poor', ...]` in
the call in `.fit`. See documentation in link [E] below.

4. Variable 'fac_differ_well' failed the non-proportional test: p-value is 0.0001.

   Advice: with so few unique values (only 2), you can include `strata=['fac_differ_well', ...]` in
the call in `.fit`. See documentation in link [E] below.

5. Variable 'fac_obstruct_Y' failed the non-proporti

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 5000 total observations, 2772 right-censored observations>
         test_name = proportional_hazard_test

---
                          test_statistic      p  -log2(p)
fac_adhere_Y        km              0.64   0.43      1.23
                    rank            0.64   0.42      1.24
fac_differ_moderate km              0.40   0.53      0.93
                    rank            0.40   0.52      0.93
fac_differ_poor     km            123.59 <0.005     92.97
                    rank          123.72 <0.005     93.06
fac_differ_well     km              7.37   0.01      7.24
                    rank            7.37   0.01      7.24
fac_node4_Y         km            122.38 <0.005     92.08
                    rank          122.49 <0.005     92.16
fac_obstruct_Y      km             37.63 <0.005     30.12
                    rank           37.70 <0.005     30.17
fac_perfor_Y        km              0.70   0.40      1.31
                    rank            0.70   0.40      1.31
fac_rx_Lev+5FU      km              1.05   0.31      1.71
                    rank            1.05   0.31      1.71
fac_rx_Obs          km              3.92   0.05      4.39
                    rank            3.88   0.05      4.35
fac_sex_M           km              0.61   0.43      1.20
                    rank            0.61   0.44      1.20
num_age             km             84.33 <0.005     64.37
                    rank           84.24 <0.005     64.31
num_nodes           km             16.14 <0.005     14.05
                    rank           16.19 <0.005     14.09
pid                 km            353.27 <0.005    259.39
                    rank          353.26 <0.005    259.39



1. Variable 'fac_differ_poor' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['fac_differ_poor', ...]` in
the call in `.fit`. See documentation in link [E] below.

2. Variable 'fac_differ_well' failed the non-proportional test: p-value is 0.0066.

   Advice: with so few unique values (only 2), you can include `strata=['fac_differ_well', ...]` in
the call in `.fit`. See documentation in link [E] below.

3. Variable 'fac_obstruct_Y' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['fac_obstruct_Y', ...]` in
the call in `.fit`. See documentation in link [E] below.

4. Variable 'fac_node4_Y' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['fac_node4_Y', ...]` in the
call in `.fit`. See documentation in link [E] below.

5. Variable 'num_age' failed the non-pro

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 649 total observations, 333 right-censored observations>
         test_name = proportional_hazard_test

---
                          test_statistic    p  -log2(p)
fac_adhere_Y        km              0.10 0.75      0.41
                    rank            0.10 0.75      0.42
fac_differ_moderate km              0.03 0.86      0.22
                    rank            0.04 0.84      0.25
fac_differ_poor     km              2.31 0.13      2.96
                    rank            2.36 0.12      3.01
fac_differ_well     km              0.25 0.62      0.69
                    rank            0.24 0.62      0.69
fac_node4_Y         km              6.44 0.01      6.48
                    rank            6.47 0.01      6.51
fac_obstruct_Y      km              3.00 0.08      3.59
                    rank            3.31 0.07      3.86
fac_perfor_Y        km              0.61 0.43      1.21
                    rank            0.66 0.42      1.26
fac_rx_Lev+5FU      km              0.09 0.76      0.39
                    rank            0.10 0.75      0.41
fac_rx_Obs          km              0.97 0.33      1.62
                    rank            1.04 0.31      1.70
fac_sex_M           km              0.28 0.60      0.74
                    rank            0.22 0.64      0.65
num_age             km              2.98 0.08      3.57
                    rank            3.05 0.08      3.63
num_nodes           km              1.38 0.24      2.06
                    rank            1.38 0.24      2.06
pid                 km              0.58 0.45      1.17
                    rank            0.59 0.44      1.18



1. Variable 'fac_node4_Y' failed the non-proportional test: p-value is 0.0110.

   Advice: with so few unique values (only 2), you can include `strata=['fac_node4_Y', ...]` in the
call in `.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-varying-covariates
[D]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Modify-the-functional-form
[E]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Stratification



In [6]:
gan_rsf_model, gan_rsf_function, gan_hazard, gan_feature_importances  = run_rsf_model(gan, test)
vae_rsf_model, vae_rsf_function, vae_hazard, vae_feature_importances = run_rsf_model(vae, test)
rsf_model, rsf_function, hazard, feature_importance = run_rsf_model(real, test)

In [ ]:
save_rsf_model_output(gan_rsf_function, hazard, gan_feature_importances, "../outputs/model_execution_outputs", "dataset_name")
save_rsf_model_output(vae_rsf_function, hazard, vae_feature_importances, "../outputs/model_execution_outputs", "dataset_name")
save_rsf_model_output(rsf_function, hazard, feature_importance, "../outputs/model_execution_outputs", "dataset_name")

In [7]:
gan_metrics = run_metrics(function_data_gan[0], gan_rsf_function, function_data_gan[1], function_data_gan[2], function_data_gan[3], function_data_gan[4])
vae_metrics = run_metrics(function_data_vae[0], vae_rsf_function, function_data_vae[1], function_data_vae[2], function_data_vae[3],function_data_vae[4])
real_metrics = run_metrics(function_data[0], rsf_function, function_data[1], function_data[2], function_data[3], function_data[4])

C:\Users\johan\anaconda3\envs\reseach_project_models_env\Lib\site-packages\SurvivalEVAL\Evaluator.py:60: UserWarning: The first time coordinate is not 0. A authentic survival curve should start from 0 with 100% survival probability. Adding 0 to the beginning of the time coordinates and 1 to the beginning of the predicted curves.
  warnings.warn("The first time coordinate is not 0. A authentic survival curve should start from 0 "


AttributeError: 'numpy.ndarray' object has no attribute 'x'

In [8]:
plot_cox_coefficients(gan_cox_model)
plot_cox_coefficients(vae_cox_model)
plot_cox_coefficients(cox_model)

In [9]:
plot_coefficients_rsf(gan_rsf_model,10)
plot_coefficients_rsf(vae_rsf_model,10)
plot_coefficients_rsf(rsf_model,10)

In [10]:
# plot_model_residuals()

In [11]:
cross_validate_and_plot()